In [3]:
import sys,os
import numpy as np
import xml.etree.ElementTree as ET
from lxml import etree
import mujoco as mj
import platform
import subprocess
import pygments
from IPython.display import HTML, display
from copy import deepcopy
import colorsys
sys.path.append('../../package/kinematics_helper/')
sys.path.append('../../package/mujoco_helper/')
sys.path.append('../../package/utility/')
sys.path.append('../../asset/')
from mujoco_parser import *
from utils import *

if platform.system() == "Linux":
    os.environ["__GLX_VENDOR_LIBRARY_NAME"] = "nvidia"
    os.environ["__NV_PRIME_RENDER_OFFLOAD"] = "1"
    os.environ["__VK_LAYER_NV_optimus"] = "NVIDIA_only"

print(f"MUJOCO_VERSION: {MUJOCO_VERSION}")

ModuleNotFoundError: No module named 'mujoco_parser'

In [ ]:
def is_macos_dark_theme():
    if sys.platform == 'darwin':
        try:
            result = subprocess.run(
                ['defaults', 'read', '-g', 'AppleInterfaceStyle'],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            return "Dark" in result.stdout
        except Exception:
            return False
    return False

def is_linux_dark_theme():
    if sys.platform.startswith('linux'):
        try:
            result = subprocess.run(
                ['gsettings', 'get', 'org.gnome.desktop.interface', 'gtk-theme'],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            theme = result.stdout.strip().strip("'")
            return "dark" in theme.lower()
        except Exception:
            return False
    return False

def is_dark_theme():
    import sys
    if sys.platform == 'darwin':
        return is_macos_dark_theme()
    elif sys.platform.startswith('linux'):
        return is_linux_dark_theme()
    else:
        return False
    
def print_xml(xml_input,color=True):
    if isinstance(xml_input, ET.Element):
        rough_string = ET.tostring(xml_input, encoding='unicode')
    else:
        rough_string = xml_input
    if color: 
        print_style = 'monokai' if is_dark_theme() else 'lovelace'
        formatter = pygments.formatters.HtmlFormatter(style=print_style)
        lexer = pygments.lexers.XmlLexer()
        highlighted = pygments.highlight(rough_string, lexer, formatter)
        display(HTML(f"<style>{formatter.get_style_defs()}</style>{highlighted}"))
    else:
        parser = etree.XMLParser(remove_blank_text=True)
        tree = etree.fromstring(rough_string, parser=parser)
        pretty_xml = etree.tostring(tree, pretty_print=True, encoding='unicode')
        print(pretty_xml)

def get_rainbow_color(i, n):
        """
        i: 현재 인덱스 (0 <= i < n)
        n: 전체 개수
        """
        # HSV 공간에서 Hue를 0 ~ 1 사이로 분산
        h = i / n
        s = 1.0    # 채도
        v = 1.0    # 명도

        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        return [r, g, b, 1.0]  # RGBA (알파=1.0)

# MjSpec
기본적인 기능을 위한 tutorial


**print_xml의 경우 색 있는 버전이 싫으시다면 color=False로 해주세요** $\rightarrow$ color 버전은 html 형식이라 복사가 어렵습니다(그래도 이쁘잖아요..ㅎㅎ)


tutorial을 skip하시려면 tutorial=False로 해주세요

In [ ]:
color=True
tutorial = True

## Tutorial

### How to call MjSpec

In [ ]:
if tutorial:
    # to call an empty mjspec:
    spec = mj.MjSpec()
    print_xml(spec.to_xml(),color=color)

In [ ]:
if tutorial:
  # or from some string
  arena_xml = """
  <mujoco>
    <visual>
      <headlight diffuse=".5 .5 .5" specular="1 1 1"/>
      <global offwidth="2048" offheight="1536"/>
      <quality shadowsize="8192"/>
    </visual>

    <asset>
      <texture type="skybox" builtin="gradient" rgb1="1 1 1" rgb2="1 1 1" width="10" height="10"/>
      <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="1 1 1" rgb2="1 1 1" markrgb="0 0 0" width="400" height="400"/>
      <material name="groundplane" texture="groundplane" texrepeat="45 45" reflectance="0"/>
    </asset>

    <worldbody>
      <geom name="floor" size="150 150 0.1" type="plane" material="groundplane"/>
    </worldbody>
  </mujoco>
  """

  spec = mj.MjSpec.from_string(arena_xml)
  print_xml(spec.to_xml(),color=color)

In [ ]:
if tutorial:
    # or from some path
    path = '../../asset/floor/floor_isaac_style.xml'
    spec = mj.MjSpec.from_file(path)
    print_xml(spec.to_xml(),color=color)

### Add some body, geom, site, ...

In [ ]:
if tutorial:
    path = '../../asset/floor/floor_isaac_style.xml'
    spec = mj.MjSpec.from_file(path)
    body = spec.worldbody.add_body( # 이런식으로 특정 spec의 object를 변수로 저장할 수도 있고(변수로 받지 않아도 됨)
        name="base_link",
        pos = np.zeros((3))
    )

    print(f'is body same as spec.body("base_link"): {body == spec.body("base_link")}') # 이름으로도 spec 검색 가능
    body.add_geom(
        name="base_link_geom",
        pos = [0.0,0.0,0.1],
        type=mj.mjtGeom.mjGEOM_BOX,
        size = [0.1, 0.1 ,0.1],
        rgba=[0,1,0,1]
    )

    spec.body("base_link").add_site(
        name="base_link_site"
    )

    body2 = body.add_body(
        name="link1",
        pos = [0.0,0.0,0.3],
    )
    body2.add_geom(
        name="link1_geom",
        pos = [0.0,0.0,0.1],
        type=mj.mjtGeom.mjGEOM_SPHERE,
        size = [0.1, 0.0, 0.0],
        rgba=[1,0,0,1]
    )

    body2.add_frame(pos=[0,0,0])
    spec.compile()

    env= MuJoCoParserClass(name='example',xml_string=spec.to_xml(),verbose=True)
    env.init_viewer()
    env.reset()
    env.forward()
    while env.is_viewer_alive():
        if env.loop_every(tick_every=5):#tick_every=1 ~ real time
            env.plot_time()
            env.render()

    # Close
    env.close_viewer()
    print_xml(spec.to_xml(),color=color)

is body same as spec.body("base_link"): True

-----------------------------------------------------------------------------
name:[example] dt:[0.002] HZ:[500]
 n_qpos:[0] n_qvel:[0] n_qacc:[0] n_ctrl:[0]
 integrator:[EULER]

n_body:[3]
 [0/3] [world] mass:[0.00]kg
 [1/3] [base_link] mass:[8.00]kg
 [2/3] [link1] mass:[4.19]kg
body_total_mass:[12.19]kg

n_geom:[3]
geom_names:['floor', 'base_link_geom', 'link1_geom']

n_joint:[0]

n_dof:[0] (=number of rows of Jacobian)

Free joint information. n_free_joint:[0]

Revolute joint information. n_rev_joint:[0]

Prismatic joint information. n_pri_joint:[0]

Control information. n_ctrl:[0]

Camera information. n_cam:[0]

n_sensor:[0]
sensor_names:[]
n_site:[1]
site_names:['base_link_site']
-----------------------------------------------------------------------------
env:[example] reset
env:[example] initalize viewer
env:[example] reset


### Recompile - changing variables during render

In [ ]:
if tutorial:
    path = '../../asset/floor/floor_isaac_style.xml'
    spec = mj.MjSpec.from_file(path)
    body = spec.worldbody.add_body( # 이런식으로 특정 spec의 object를 변수로 저장할 수도 있고(변수로 받지 않아도 됨)
        name="base_link",
        pos = np.zeros((3))
    )

    print(f'is body same as spec.body("base_link"): {body == spec.body("base_link")}') # 이름으로도 spec 검색 가능
    body.add_geom(
        name="base_link_geom",
        pos = [0.0,0.0,0.1],
        type=mj.mjtGeom.mjGEOM_BOX,
        size = [0.1, 0.1 ,0.1],
        rgba=[0,1,0,1]
    )

    body2 = body.add_body(
        name="link1",
        pos = [0.0,0.0,0.3],
    )
    body2.add_geom(
        name="link1_geom",
        pos = [0.0,0.0,0.1],
        type=mj.mjtGeom.mjGEOM_SPHERE,
        size = [0.1, 0.0, 0.0],
        rgba=[1,0,0,1]
    )

    spec.compile()

    env= MuJoCoParserClass(name='example',xml_string=spec.to_xml(),verbose=True)
    env.init_viewer()
    env.reset()
    env.forward()
    env.render()
    i=0.01
    while env.is_viewer_alive():
        if env.loop_every(tick_every=10):
            spec.geom("link1_geom").size=[i,0,0]
            spec.geom("link1_geom").rgba = get_rainbow_color(i%2,2)
            env.model,env.data = spec.recompile(env.model,env.data)
            env.forward(increase_tick=False)
            env.viewer.model = env.model
            env.viewer.data = env.data
            i+=0.01
        env.forward()
        if env.loop_every(tick_every=5):#tick_every=1 ~ real time
            env.plot_time()
            env.render()





    # Close
    env.close_viewer()
    print_xml(spec.to_xml(),color=color)

is body same as spec.body("base_link"): True

-----------------------------------------------------------------------------
name:[example] dt:[0.002] HZ:[500]
 n_qpos:[0] n_qvel:[0] n_qacc:[0] n_ctrl:[0]
 integrator:[EULER]

n_body:[3]
 [0/3] [world] mass:[0.00]kg
 [1/3] [base_link] mass:[8.00]kg
 [2/3] [link1] mass:[4.19]kg
body_total_mass:[12.19]kg

n_geom:[3]
geom_names:['floor', 'base_link_geom', 'link1_geom']

n_joint:[0]

n_dof:[0] (=number of rows of Jacobian)

Free joint information. n_free_joint:[0]

Revolute joint information. n_rev_joint:[0]

Prismatic joint information. n_pri_joint:[0]

Control information. n_ctrl:[0]

Camera information. n_cam:[0]

n_sensor:[0]
sensor_names:[]
n_site:[0]
site_names:[]
-----------------------------------------------------------------------------
env:[example] reset
env:[example] initalize viewer
env:[example] reset


# Manipulator

In [ ]:
class JointClass(object):
    def __init__(self,name="joint",rel_pos=np.array([0.0,0.0,0.0]),axis=np.array([0,0,1]),r_cylinder = 0.03,rgba=np.array([0.5,0.5,0.5,1]),r_link = 0.03):
        self.name=name
        self.rel_pos=rel_pos
        self.axis=axis
        self.r_cylinder=r_cylinder
        self.rgba=rgba
        self.r_link = r_link

In [ ]:
def set_actuator(spec,joint_name,kp=0,kv=0,timeconst=0,gain_type="fixed",
                    bias_type="affine",trn_type="joint"
                    ,ctrlrange=None,forcerange=None,auto_ctrl=False, auto_force=True,verbose=False):
    """
        add_exclude: name, bodyname1, bodyname2, info.
        mjsBody.mjs_firstChild()로 코드를 구성하고 싶지만 python에는 아직 구현이 안된 듯하다
    """
    gainprm=[kp,0,0]
    biasprm=[0,-kp,-kv]
    dynprm=[timeconst,0,0,0,0,0,0,0,0,0]
    gainprm=[kp,0,0,0,0,0,0,0,0,0]
    biasprm=[0,-kp,-kv,0,0,0,0,0,0,0]
    if trn_type=="joint":
        trntype=mj.mjtTrn.mjTRN_JOINT
    if trn_type=="jointparent":
        trntype=mj.mjtTrn.mjTRN_JOINTPARENT
    if trn_type=="tendon":
        trntype=mj.mjtTrn.mjTRN_TENDON
    if trn_type=="site":
        trntype=mj.mjtTrn.mjTRN_SITE

    if timeconst==0:
        dyntype=mj.mjtDyn.mjDYN_NONE
    elif timeconst==1:
        dyntype=mj.mjtDyn.mjDYN_FILTEREXACT
    if gain_type=="fixed":
        gaintype=mj.mjtGain.mjGAIN_FIXED
    elif gain_type =="affine":
        gaintype=mj.mjtGain.mjGAIN_AFFINE
    
    if bias_type=="affine":
        biastype=mj.mjtBias.mjBIAS_AFFINE
    elif bias_type=="none":
        biastype=mj.mjtBias.mjBIAS_NONE
    """Valid options are:
    name, gaintype, gainprm, biastype, biasprm, dyntype,
    dynprm, actdim, actearly, trntype, gear, target, refsite,
    slidersite, cranklength, lengthrange, inheritrange, ctrllimited,
    ctrlrange, forcelimited, forcerange, actlimited, actrange, group,
    userdata, plugin, info."""
    if ctrlrange is not None:
        ctrllimited=1
    

    if forcerange is not None:
        forcelimited=1
    if auto_ctrl:
        ctrllimited=2
    if auto_force:
        forcelimited=2
    spec.add_actuator(name=joint_name+'_act',dyntype=dyntype,trntype=trntype,
                        target=joint_name,gaintype=gaintype,biastype=biastype,
                        dynprm=dynprm,gainprm=gainprm,biasprm=biasprm,
                        ctrllimited=ctrllimited,
                        ctrlrange=ctrlrange,
                        forcelimited=forcelimited,
                        # forcerange=forcerange,
                        )

In [ ]:
def axis_to_quat(axis):
            axis = np.array(axis)
            axis = axis / np.linalg.norm(axis)
            # z-axis를 axis로 회전시켜주는 quaternion
            quat = np.zeros(4)
            mj.mju_quatZ2Vec(quat, axis)
            return quat

def fix_joint_and_act_limits(spec):
    joints=spec.worldbody.find_all(mj.mjtObj.mjOBJ_JOINT)
    
    for i in range(len(joints)):
        joint = joints[i]
        print(f"name of joint: {joint.name}, range: {joint.range}")
        if joint.range is None or (joint.range[0] == joint.range[1]):
            joint.limited = 0
            joint.range = [0.0, 0.0]  # 그냥 명시적으로 넣어주는 것도 OK
    actuators = spec.actuators
    for j in range(len(actuators)):
        act = actuators[j]
        if act.ctrlrange is None or (act.ctrlrange[0]==act.ctrlrange[1]):
            act.ctrllimited=1
            jnt_range = spec.joint(act.target).range
            if jnt_range[0]==jnt_range[1]:
                act.ctrlrange=[-np.pi,np.pi]
            else:
                act.ctrlrange = jnt_range

def build_manipulator(
        joints=[],
        robot_name="manipulator",
        collision_geom = True, 
        init_xml_path = None,
        spec_to_reuse = None,
        collision_margin = 0.01,
        kp=10,
        kv=10,
        armature=0.1,
        damping=5,
        ):
    if init_xml_path is not None:
        spec = mj.MjSpec.from_file(init_xml_path)
        spec.modelname = robot_name+'_scene'
    elif init_xml_path is None and spec_to_reuse is not None:
        spec = spec_to_reuse
    else:
        spec = mj.MjSpec()
        spec.modelname=robot_name
        spec.add_default('visual',spec.default)
        visual=spec.find_default('visual')
        # visual.geom.type=mj.mjtGeom.mjGEOM_ELLIPSOID
        visual.geom.contype=0
        visual.geom.conaffinity=0
        visual.geom.group=1
    is_visual = spec.find_default('visual')
    if is_visual is not None:
        pass
    else:
        spec.add_default('visual',spec.default)
        visual=spec.find_default('visual')
        # visual.geom.type=mj.mjtGeom.mjGEOM_ELLIPSOID
        visual.geom.contype=0
        visual.geom.conaffinity=0
        visual.geom.group=1
    if collision_geom:
        if spec_to_reuse is not None or init_xml_path is not None:
            is_collision = spec.find_default('collision')
            if is_collision is not None:
                pass
            else:
                spec.add_default('collision',spec.default)
                collision = spec.find_default('collision')
                collision.geom.group=2
        else:
            spec.add_default('collision',spec.default)
            collision = spec.find_default('collision')
            collision.geom.group=2
    collision_group = spec.find_default('collision').geom.group
    visual_group = spec.find_default('visual').geom.group
    for i,joint in enumerate(joints):
        if joint.name !='joint' and joint.name is not None:
            body_name=joint.name
        elif joint.name=='joint':
            body_name = joint.name+'_'+i
        else:
            body_name = 'joint_'+i
        if i==0:
            parent_body = spec.worldbody
            parent = JointClass(name="joint")
        else:
            parent_body = spec.body(parent.name+'_body')
        visual_color = joint.rgba
        if collision:
            collision_color = np.concatenate([joint.rgba[0:3], [0.3]])
        is_same = np.allclose(joint.rel_pos, np.array([0.0,0.0,0.0]))
        if body_name =="end_effector":
            frame1 = parent_body.add_frame(
                name='attach_here',
                quat = axis_to_quat(joint.axis)
            )
            # body = parent_body.add_body(name=body_name,pos=np.array([0.0,0.0,0.0]))
            spec2 = mj.MjSpec()
            body = spec2.worldbody.add_body(name=body_name, pos = np.array([0.0,0.0,0.0]))
            frame2 = body.add_frame(
                name='to_attach',
            )
            # target
            body.add_geom(
                name='target',
                pos = joint.rel_pos,
                type=mj.mjtGeom.mjGEOM_BOX,
                size=np.array([joint.r_cylinder/2,joint.r_cylinder/2,joint.r_cylinder/2]),
                group = visual_group,
                rgba = visual_color
            )
            #end_effector 뼈대
            body.add_geom(
                name='geom_collision',
                # pos=np.where(np.arange(3) == idx, joint.r_cylinder/2, 0),
                type=mj.mjtGeom.mjGEOM_BOX,
                # size= np.where(np.arange(3) == idx, joint.r_link, joint.r_cylinder),
                size = np.array([joint.r_link*2,joint.r_cylinder,joint.r_cylinder]),
                # quat = axis_to_quat(joint.axis),
                group = collision_group,
                rgba = visual_color
            )
            grip_1 = body.add_body(
                name='gripper_1',
                pos = np.array([joint.r_link,joint.r_cylinder/2,0.0]),
            )
            grip_jnt1 = grip_1.add_joint(
                name='gripper_1_slide_joint',
                type=mj.mjtJoint.mjJNT_SLIDE,
                axis = np.array([1,0,0]),
                range = np.array([-joint.r_link+joint.r_cylinder/2,joint.r_link-joint.r_cylinder/2]),
                armature=armature,
                damping=damping,
                limited=1,
            )
            
            grip_1.add_geom(
                name='gripper_1_geom',
                pos = np.array([0.0,((joint.rel_pos-joint.r_cylinder+collision_margin)/2)[1],0.0]),
                size = np.array([joint.r_cylinder/2,((joint.rel_pos-joint.r_cylinder)/2)[1]+collision_margin,joint.r_cylinder]),
                type = mj.mjtGeom.mjGEOM_BOX,
                group = collision_group,
                rgba = visual_color
            )
            grip_2 = body.add_body(
                name='gripper_2',
                pos = np.array([-joint.r_link,joint.r_cylinder/2,0.0]),
            )
            grip_jnt2=grip_2.add_joint(
                name='gripper_2_slide_joint',
                type=mj.mjtJoint.mjJNT_SLIDE,
                axis = np.array([1,0,0]),
                range = np.array([-joint.r_link+joint.r_cylinder/2,joint.r_link-joint.r_cylinder/2]),
                armature=armature,
                damping=damping,
                limited=1,
            )
            grip_2.add_geom(
                name='gripper_2_geom',
                pos = np.array([0.0,((joint.rel_pos-joint.r_cylinder+collision_margin)/2)[1],0.0]),
                size = np.array([joint.r_cylinder/2,((joint.rel_pos-joint.r_cylinder)/2)[1]+collision_margin,joint.r_cylinder]),
                type = mj.mjtGeom.mjGEOM_BOX,
                group = collision_group,
                rgba = visual_color
            )
            
            spec.copy_during_attach = True
            frame1.attach_body(body,body_name+'_','')
            spec.add_equality(
                name="equality_gripper",
                type=mj.mjtEq.mjEQ_JOINT,
                active=1,
                name1=body_name+'_gripper_1_slide_joint',
                name2=body_name+'_gripper_2_slide_joint',
                data=[0, -1, 0, 0, 0]+[0]*6
            )
            set_actuator(spec=spec,joint_name=body_name+'_gripper_1_slide_joint',kp=kp,kv=kv,auto_ctrl=False,ctrlrange=grip_jnt1.range)
            spec.add_exclude(
                bodyname1=body_name+"_gripper_1",bodyname2=body_name+"_gripper_2"
            )

            continue
        
        body = parent_body.add_body(
            name=body_name+'_body',
            pos=joint.rel_pos,
        )
        body.add_geom(
            name=body_name+'_geom_visual',
            # pos=np.array([0,0,0]),
            type=mj.mjtGeom.mjGEOM_CYLINDER,
            size= np.array([joint.r_cylinder,joint.r_cylinder,joint.r_cylinder]),
            quat = axis_to_quat(joint.axis),
            group = visual_group,
            rgba = visual_color
        )
        
        if not is_same:
            parent_body.add_geom(
                name=body_name+'_link_geom_visual',
                type=mj.mjtGeom.mjGEOM_ELLIPSOID,
                size = np.array([joint.r_link,joint.r_link,joint.r_link]),
                fromto = np.concatenate([np.array([0.0,0.0,0.0]),joint.rel_pos]),
                rgba = visual_color,
            )
        if collision:
            body.add_geom(
                name=body_name+'_geom_collision',
                # pos=np.array([0,0,0]),
                type=mj.mjtGeom.mjGEOM_SPHERE,
                size= np.array([joint.r_cylinder+collision_margin,0.0,0.0]),
                # quat = axis_to_quat(joint.axis),
                group = collision_group,
                rgba = collision_color,
            )
            if not is_same:
                ratio = (collision_margin+parent.r_cylinder)/np.linalg.norm(joint.rel_pos)
                if ratio>1:
                    raise ValueError(f"ratio is larger than 1: {ratio}")
                elipsoid_start = joint.rel_pos*ratio
                elipsoid_end = joint.rel_pos*(1-ratio)
                parent_body.add_geom(
                    name=body_name+'_link_geom_collision',
                    type=mj.mjtGeom.mjGEOM_CAPSULE,
                    size = np.array([joint.r_link,joint.r_link,joint.r_link]),
                    fromto = np.concatenate([elipsoid_start,elipsoid_end]),
                    rgba = collision_color,
                )
        jnt = body.add_joint(
            name=body_name,
            type = mj.mjtJoint.mjJNT_HINGE,
            axis=joint.axis,
            armature=armature,
            damping=damping,
            limited=1,
        )
        set_actuator(spec=spec,joint_name=body_name,kp=kp,kv=kv,ctrlrange=jnt.range, auto_ctrl=False)
        if i==0:
            spec.add_exclude(
                bodyname1="world",bodyname2=body_name+'_body'
        )
        parent = deepcopy(joint)
    fix_joint_and_act_limits(spec)
    spec.compile()
    return spec.to_xml()


        
        

In [ ]:
spec_try = mj.MjSpec()

In [ ]:
body=spec_try.worldbody.add_body(name="try")
body.add_joint(
    name="try_joint",
    axis=[0.0,1.0,0.0],
    type=mj.mjtJoint.mjJNT_HINGE,
    armature=0.1,
    damping=5,
)
body.add_geom(
    name="try_geom",
    type=mj.mjtGeom.mjGEOM_BOX,
    size=[0.1,0.1,0.1],
    pos=[0,0,3]
)

In [ ]:
print_xml(spec_try.to_xml())

In [ ]:

joint_1 = JointClass(
    name="joint1",
    rel_pos=np.array([0.0,0.0,0.0]),
    axis=np.array([0,0,1]),
    r_cylinder=0.03, 
    rgba = get_rainbow_color(0,6),
    )
joint_2 = JointClass(
    name="joint2",
    rel_pos=np.array([0.0,0.0,0.3]),
    axis=np.array([1,0,0]),
    r_cylinder=0.03, 
    rgba = get_rainbow_color(1,6),
    )
joint_3 = JointClass(
    name="joint3",
    rel_pos=np.array([0.0,0.2,0.4]),
    axis=np.array([1,0,0]),
    r_cylinder=0.03, 
    rgba = get_rainbow_color(2,6),
    )
joint_4 = JointClass(
    name="joint4",
    rel_pos=np.array([0.0,0.3,0.0]),
    axis=np.array([1,0,0]),
    r_cylinder=0.03, 
    rgba = get_rainbow_color(3,6),
    )
joint_5 = JointClass(
    name="joint5",
    rel_pos=np.array([0.0,0.3,0.0]),
    axis=np.array([0,1,0]),
    r_cylinder=0.03, 
    rgba = get_rainbow_color(4,6),
    )
joint_6 = JointClass(
    name="end_effector",
    rel_pos = np.array([0.0,0.3,0.0]), # end_effector target의 위치 - 꼭 y좌표에만 작성해주세요
    axis = np.array([0,0,1]), # end_effector_target의 위쪽이 어디를 바라보는지
    r_cylinder = 0.03, # end_effector 두께
    r_link=0.1, # end_effector 너비/2
    rgba = get_rainbow_color(5,6)
)
joints=[joint_1, joint_2, joint_3, joint_4, joint_5,joint_6]
xml= build_manipulator(
    joints=joints, 
    collision_geom = True, 
    init_xml_path = '../../asset/floor/floor_isaac_style.xml',
    kp=200,
    kv=10,
    )
print_xml(xml,color=False)

name of joint: joint1, range: [0. 0.]
name of joint: joint2, range: [0. 0.]
name of joint: joint3, range: [0. 0.]
name of joint: joint4, range: [0. 0.]
name of joint: joint5, range: [0. 0.]
name of joint: end_effector_gripper_1_slide_joint, range: [-0.085  0.085]
name of joint: end_effector_gripper_2_slide_joint, range: [-0.085  0.085]
<mujoco model="manipulator_scene">
  <compiler angle="radian"/>
  <visual>
    <headlight diffuse="0.6 0.6 0.6" specular="0.9 0.9 0.9"/>
    <rgba haze="0.15 0.25 0.35 1"/>
  </visual>
  <default>
    <default class="visual">
      <geom contype="0" conaffinity="0" group="1"/>
    </default>
    <default class="collision">
      <geom group="2"/>
    </default>
    <default class="end_effector_main"/>
  </default>
  <asset>
    <texture type="skybox" builtin="gradient" rgb1="0.4 0.5 0.9" rgb2="0 0 0" width="512" height="3072"/>
    <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.9 0.9 0.9" rgb2="0.8 0.8 0.8" markrgb="0.8 0.8 0

In [ ]:
model = mj.MjModel.from_xml_string(xml)
data = mj.MjData(model)
mj.mj_forward(model,data)
# joint limit 가져오기
for i in range(model.njnt):
    qpos_range = model.jnt_range[i]
    name = mj.mj_id2name(model,mj.mjtObj.mjOBJ_JOINT,i)
    print(f"Joint {name}: limit = {qpos_range}")

Joint joint1: limit = [0. 0.]
Joint joint2: limit = [0. 0.]
Joint joint3: limit = [0. 0.]
Joint joint4: limit = [0. 0.]
Joint joint5: limit = [0. 0.]
Joint end_effector_gripper_1_slide_joint: limit = [-0.085  0.085]
Joint end_effector_gripper_2_slide_joint: limit = [-0.085  0.085]


In [ ]:
env= MuJoCoParserClass(name='example',xml_string=xml,verbose=True)
# env.open_interactive_viewer()
init_ctrl = env.get_ctrl(ctrl_names=env.ctrl_names)
sliders = MultiSliderClass(
    n_slider      = env.n_ctrl,
    title         = 'Sliders for [%s] Control'%(env.name),
    window_width  = 600,
    window_height = 350,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 300,
    label_texts   = env.ctrl_names,
    slider_mins   = env.ctrl_mins,
    slider_maxs   = env.ctrl_maxs,
    slider_vals   = init_ctrl,
    verbose       = False,
)
# Arrange objects
env.reset(step=True)
obj_names = env.get_body_names(prefix='obj_')
n_obj = len(obj_names)
obj_xyzs = sample_xyzs(
    n_sample  = n_obj,
    x_range   = [+0.6,+1.0],
    y_range   = [-0.45,+0.45],
    z_range   = [0.8,0.81],
    min_dist  = 0.2,
    xy_margin = 0.0
)
for obj_idx in range(n_obj):
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=np.eye(3,3))
env.set_geom_color(body_names_to_color=obj_names,rgba_list=get_colors(n_obj))
# Loop
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    # Update
    sliders.update()
    env.step(ctrl=sliders.get_slider_values())
    # Render
    if env.loop_every(tick_every=50):
        env.plot_time()
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_links_between_bodies(rgba=(0,0,0,1),r=0.001) # link information
        env.plot_contact_info()
        env.render()
# Close slider
sliders.close()
print ("Done.")


-----------------------------------------------------------------------------
name:[example] dt:[0.002] HZ:[500]
 n_qpos:[7] n_qvel:[7] n_qacc:[7] n_ctrl:[6]
 integrator:[EULER]

n_body:[9]
 [0/9] [world] mass:[0.00]kg
 [1/9] [joint1_body] mass:[1.74]kg
 [2/9] [joint2_body] mass:[2.43]kg
 [3/9] [joint3_body] mass:[1.74]kg
 [4/9] [joint4_body] mass:[1.74]kg
 [5/9] [joint5_body] mass:[0.44]kg
 [6/9] [end_effector_end_effector] mass:[1.47]kg
 [7/9] [end_effector_gripper_1] mass:[0.52]kg
 [8/9] [end_effector_gripper_2] mass:[0.52]kg
body_total_mass:[10.60]kg

n_geom:[23]
geom_names:['floor', 'joint1_geom_visual', 'joint1_geom_collision', 'joint2_link_geom_visual', 'joint2_link_geom_collision', 'joint2_geom_visual', 'joint2_geom_collision', 'joint3_link_geom_visual', 'joint3_link_geom_collision', 'joint3_geom_visual', 'joint3_geom_collision', 'joint4_link_geom_visual', 'joint4_link_geom_collision', 'joint4_geom_visual', 'joint4_geom_collision', 'joint5_link_geom_visual', 'joint5_link_geom_